# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.

%load_ext dotenv
%dotenv 

In [31]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [13]:
import os
from glob import glob

# Write your code below.

import pandas as pd
import numpy as np
ft_path = os.getenv("PRICE_DATA")
#df_raw = pd.read_csv(ft_path)
#os.makedirs(ft_path, exist_ok=True)

parquet_files = glob(os.path.join(ft_path, "*/*/part.0.parquet"), recursive=True)

#parquet_files = [f for f in glob(os.path.join(ft_path, "*/*.parquet")) if os.path.isfile(f)]


In [ ]:
print(parquet_files)

In [33]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

In [ ]:
dd_px.head()

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [34]:
# Write your code below.

import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1), Adj_Close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))


C:\Users\Tooman\AppData\Local\Temp\ipykernel_19728\3821611733.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [35]:
dd_feat.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
HUM,2008-01-02 00:00:00+00:00,69.752991,78.120003,79.139999,76.510002,76.510002,2224700,Health Care,Managed Health Care,2008,NaN,NaN,NaN,2.629997
HUM,2008-01-03 00:00:00+00:00,71.985252,80.620003,81.120003,77.809998,78.489998,2426800,Health Care,Managed Health Care,2008,78.120003,69.752991,0.032002,3.310005
HUM,2008-01-04 00:00:00+00:00,70.958427,79.470001,81.110001,78.949997,80.099998,1619200,Health Care,Managed Health Care,2008,80.620003,71.985252,-0.014264,2.160004
HUM,2008-01-07 00:00:00+00:00,72.824547,81.559998,81.610001,79.110001,80.110001,1984500,Health Care,Managed Health Care,2008,79.470001,70.958427,0.026299,2.500000
HUM,2008-01-08 00:00:00+00:00,73.146004,81.919998,83.070000,81.470001,82.040001,2468000,Health Care,Managed Health Care,2008,81.559998,72.824547,0.004414,1.599998


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [36]:
# Write your code below.
# convert dask dataframe to pandas dataframe
dd_pd = dd_feat.compute()

In [37]:
dd_pd = dd_pd.groupby('ticker', group_keys=False).apply(lambda x: x.assign(rolling_avg_return = x['returns'].rolling(window=10).mean()))


In [39]:
dd_pd.head(100)

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag_1,Adj_Close_lag,returns,hi_lo_range,rolling_avg_return
ticker,,,,,,,,,,,,,,,
HUM,2024-01-02 00:00:00+00:00,466.467102,470.100006,471.549988,458.149994,459.739990,1966900,Health Care,Managed Health Care,2024,NaN,NaN,NaN,13.399994,NaN
HUM,2024-01-03 00:00:00+00:00,460.930237,464.519989,475.000000,463.839996,474.910004,1307400,Health Care,Managed Health Care,2024,470.100006,466.467102,-0.011870,11.160004,NaN
HUM,2024-01-04 00:00:00+00:00,462.934662,466.540009,471.369995,466.170013,469.100006,951800,Health Care,Managed Health Care,2024,464.519989,460.930237,0.004349,5.199982,NaN
HUM,2024-01-05 00:00:00+00:00,454.728577,458.269989,464.750000,453.730011,464.029999,1692700,Health Care,Managed Health Care,2024,466.540009,462.934662,-0.017726,11.019989,NaN
HUM,2024-01-08 00:00:00+00:00,457.179474,460.739990,464.380005,451.989990,458.980011,1584500,Health Care,Managed Health Care,2024,458.269989,454.728577,0.005390,12.390015,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HUM,2024-05-17 00:00:00+00:00,354.347504,356.200012,356.779999,352.859985,355.600006,1435600,Health Care,Managed Health Care,2024,355.190002,353.342743,0.002844,3.920013,0.010663
HUM,2024-05-20 00:00:00+00:00,353.372620,355.220001,356.589996,353.549988,356.200012,1430400,Health Care,Managed Health Care,2024,356.200012,354.347504,-0.002751,3.040009,0.010796
HUM,2024-05-21 00:00:00+00:00,353.780487,355.630005,357.140015,353.959991,356.959991,953600,Health Care,Managed Health Care,2024,355.220001,353.372620,0.001154,3.180023,0.009220


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?


Answer

It wasn’t strictly necessary to convert to Pandas to calculate the moving average return, as Dask can handle rolling computations directly. However, which approach is better depends on the size of the data and memory constraints:

If memory is not a concern and the dataset is relatively small, Pandas can handle the task efficiently. For larger datasets that don’t fit in memory or require distributed computing, Dask is the better choice as it will optimize for memory and computation. Dask is designed to work on distributed clusters, so if you’re working with a large dataset in a distributed environment, it’s much better to stay within Dask.







(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.